In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
%matplotlib inline

import glob, os
from pathlib import *
import json

In [5]:
pwd

'/home/cabbagepatch/github/srs_spotify_bsns/notebooks'

### Getting paths for data

In [3]:
# using pathlib
BASE_DIR = Path('../').resolve() 
DATA_DIR = BASE_DIR / "data"
NB_DIR = BASE_DIR / "notebooks"

In [37]:
# putting json files in sorted list
file_list = [f for f in DATA_DIR.iterdir() if f.endswith('.json')]
    
file_list.sort()    

AttributeError: 'PosixPath' object has no attribute 'endswith'

In [33]:
# viewing current list contents
for i,f in enumerate(file_list):
    print(i,f)

0 /home/cabbagepatch/github/srs_spotify_bsns/data/FamilyPlan.json
1 /home/cabbagepatch/github/srs_spotify_bsns/data/Follow.json
2 /home/cabbagepatch/github/srs_spotify_bsns/data/KidsAccount.json
3 /home/cabbagepatch/github/srs_spotify_bsns/data/Payments.json
4 /home/cabbagepatch/github/srs_spotify_bsns/data/Playlist1.json
5 /home/cabbagepatch/github/srs_spotify_bsns/data/Read Me First.pdf
6 /home/cabbagepatch/github/srs_spotify_bsns/data/SearchQueries.json
7 /home/cabbagepatch/github/srs_spotify_bsns/data/StreamingHistory0.json
8 /home/cabbagepatch/github/srs_spotify_bsns/data/StreamingHistory1.json
9 /home/cabbagepatch/github/srs_spotify_bsns/data/StreamingHistory2.json
10 /home/cabbagepatch/github/srs_spotify_bsns/data/Userdata.json
11 /home/cabbagepatch/github/srs_spotify_bsns/data/YourLibrary.json


In [31]:
# opening test json file
with open(file_list[5]) as data_file:
    data = json.load(data_file)

len(data), data

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 10: invalid start byte